# Correlate neuron activity and behavior

Load the required packages

In [ ]:
import numpy as np
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import os
import sys
import re
import math
import holoviews as hv
import panel as pn
import bisect
hv.extension('bokeh', 'matplotlib')
from IPython.display import display
from ipyfilechooser import FileChooser
import warnings
from scipy.stats import zscore
import matplotlib.cm as cm
import matplotlib.colors as mcolors
from scipy.signal import resample
from scipy.stats import sem
import json

%matplotlib widget
warnings.filterwarnings("ignore")
#%reset


def remove_outliers_avg_filter(data):
    data = np.array(data, dtype=float)  # Ensure NumPy array with float type
    filtered_data = np.copy(data)  # Copy to avoid modifying original data
    for i in range(len(data)):
        if not np.isnan(data[i]):  # Skip valid values
            continue
        # Find the closest previous non-NaN value
        prev_idx = i - 1
        while prev_idx >= 0 and np.isnan(data[prev_idx]):
            prev_idx -= 1        
        # Find the closest next non-NaN value
        next_idx = i + 1
        while next_idx < len(data) and np.isnan(data[next_idx]):
            next_idx += 1
        # Compute average if both values exist
        if prev_idx >= 0 and next_idx < len(data):
            filtered_data[i] = (data[prev_idx] + data[next_idx]) / 2
        # If neither exists, NaN remains
    return filtered_data

def find_closest_index_sorted(arr, target):
    idx = bisect.bisect_left(arr, target)  # Find the insertion point
    if idx == 0:
        return 0
    if idx == len(arr):
        return len(arr) - 1
    before = idx - 1
    after = idx
    return before if abs(arr[before] - target) <= abs(arr[after] - target) else after

# Sample callback function
def update_my_folder(chooser):
    global dpath
    dpath = chooser.selected
    %store dpath
    return 

def detect_longest_lowest_sequence(arr, margin=0):
    min_val = np.nanmin(arr)  # Find minimum value
    threshold = min_val + margin  # Define threshold based on margin    
    # Get indices where values are within the threshold
    min_indices = np.where(arr <= threshold)[0]
    # Identify consecutive sequences
    longest_sequence = None
    if len(min_indices) > 0:
        start = min_indices[0]
        max_duration = 0  # Track longest duration        
        for i in range(1, len(min_indices)):
            if min_indices[i] != min_indices[i - 1] + 1:  # Not consecutive
                duration = min_indices[i - 1] - start + 1
                if duration > max_duration:
                    max_duration = duration
                    longest_sequence = (start, min_indices[i - 1], duration)
                start = min_indices[i]  # Reset start index        
        # Check last detected sequence
        duration = min_indices[-1] - start + 1
        if duration > max_duration:
            longest_sequence = (start, min_indices[-1], duration)
    return min_val, threshold, longest_sequence


In [ ]:
minian_path = os.path.join(os.path.abspath('..'),'minian')
print("The folder used for minian procedures is : {}".format(minian_path))

sys.path.append(minian_path)
from minian.utilities import (
    TaskAnnotation,
    get_optimal_chk,
    load_videos,
    open_minian,
    save_minian,
)

Select the minian folder

In [ ]:
try: # tries to retrieve dpath either from a previous run or from a previous notebook
    %store -r dpath
except:
    print("the path was not defined in store")
    #dpath = "/Users/mb/Documents/Syntuitio/AudreyHay/PlanB/ExampleRedLines/2022_08_06/13_30_01/My_V4_Miniscope/"
    dpath = "//10.69.168.1/crnldata/waking/audrey_hay/L1imaging/AnalysedMarch2023/Gaelle/Baseline_recording"

fc1 = FileChooser(dpath,select_default=True, show_only_dirs = True, title = "<b>Folder with videos</b>")
display(fc1)
# Register callback function
fc1.register_callback(update_my_folder)

Import spatial map, Ca2+ traces

In [ ]:
sessiontype=Path(dpath).parent.parent.parent.parent.parent.name
mice=Path(dpath).parent.parent.parent.parent.name
date=Path(dpath).parent.parent.name
hour=Path(dpath).parent.name
print(sessiontype, '-', mice, '-',date,'-', hour)


minianversion = 'minian'
try: # tries to retrieve minianversion either from a previous run or from a previous notebook
    %store -r minianversion
except:
    print("the minian folder to use was not defined in store")
    minianversion = 'minian' #'minianAB' # or 'minian_intermediate'
    %store minianversion

folderMouse = Path(os.path.join(dpath,minianversion))
print(folderMouse)
minian_ds = open_minian(folderMouse)

StampsMiniscopeFile = Path(os.path.join(dpath, f'timeStamps.csv'))
tsmini=pd.read_csv(StampsMiniscopeFile)['Time Stamp (ms)']
minian_freq=round(1/np.mean(np.diff(np.array(tsmini)/1000)))
print('Miniscope sample rate =', minian_freq, 'Hz')

Ao = minian_ds['A']
Co = minian_ds['C']

TodropFile = folderMouse / f'TodropFileAB.json'
with open(TodropFile, 'r') as f:
    unit_to_drop = json.load(f)
    
C=Co.drop_sel(unit_id=unit_to_drop)
A=Ao.drop_sel(unit_id=unit_to_drop)

idloc = A.idxmax("unit_id")
Hmax = A.idxmax("height")
Hmax2 = Hmax.max("width")

Wmax = A.idxmax("width")
Wmax2 = Wmax.max("height")
coord1 = Wmax2.to_series()
coord2 = Hmax2.to_series()

a = pd.concat([coord1,coord2], axis=1)
unit = len(a)
print("{} units have been found".format(unit))

"""
filename='//10.69.168.1/crnldata/forgetting/Aurelie/CheeseboardExperiment/DAQ_data/AB/Summary_table.xlsx'
Summary_table = pd.read_excel(filename)
start_time = Summary_table.loc[(Summary_table['session_type'] == sessiontype) & (Summary_table['mice'] == mice) & (Summary_table['session_time'] == date) & (Summary_table['trial_time'] == hour), 'start_time'].item()
found_reward_time = Summary_table.loc[(Summary_table['session_type'] == sessiontype) & (Summary_table['mice'] == mice) & (Summary_table['session_time'] == date) & (Summary_table['trial_time'] == hour), 'found_reward_time'].item()
print('Start time =',start_time)
print('Reward found time =',found_reward_time)
"""

Import DeepLabCut data

In [ ]:
dlcpath=Path(f'{Path(dpath).parent}/My_first_WebCam/')
for file in os.listdir(dlcpath):
    if file.endswith(('.h5')):
        dlcfile=file
        break
dlc_path = os.path.join(dlcpath, dlcfile)
print(dlcfile)

# Load HDF5 file
df = pd.read_hdf(dlc_path)
directory = os.path.dirname(dlc_path)
timestamps_path = Path(directory,'timeStamps.csv')
if timestamps_path.exists():
    timestamps = pd.read_csv(timestamps_path)
    frame_rate = round(1/(np.mean(np.diff(timestamps.iloc[:,1]))/1000))  # fps
    print(f'Acquisition with DAQ, frame rate = {frame_rate} fps')
else:
    frame_rate = 16  # fps /!\ CHANGE ACCORDING TO YOUR DATA
    print(f'Acquisition with Webcam, frame rate = {frame_rate} fps')
with open("//10.69.168.1/crnldata/forgetting/Aurelie/CheeseboardExperiment/DAQ_data/AB/Reward_position.txt", "r") as file:
    text = file.read()  
numbers = re.findall(r"[-+]?\d*\.\d+|\d+", text)
reward_x, reward_y = map(float, numbers)

# Define parameters
pixel_to_cm = 2.25  
reward_zone = 8 * pixel_to_cm if "Training" in dlc_path else 20 * pixel_to_cm  # 8 cm for training, 20 cm for test
table_center_x, table_center_y = 313, 283  # Center of the cheeseboard table on the video
table_radius = 270 / 2
min_stay_at_reward = 5 * frame_rate  # 5 seconds

# Define functions
def calculate_relative_distance(x1, y1, x2, y2):
    return math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

def calculate_distance_run(x_coords, y_coords):
    distances = np.sqrt(np.diff(x_coords) ** 2 + np.diff(y_coords) ** 2)
    for i in range(1, len(distances) - 1):
        if np.isnan(distances[i]):
            neighbors = [distances[i-1], distances[i+1]]
            distances[i] = np.mean([x for x in neighbors if not np.isnan(x)])
    total_distance_cm = np.nansum(distances) / pixel_to_cm  # Convert to cm
    return total_distance_cm, distances

def find_long_non_nan_sequences(arr, min_length=5):
    mask = ~np.isnan(arr)  # True for non-NaN values
    diff = np.diff(np.concatenate(([0], mask.astype(int), [0])))  # Add padding to detect edges
    starts = np.where(diff == 1)[0]  # Where a sequence starts
    ends = np.where(diff == -1)[0]   # Where a sequence ends
    sequences = [arr[start:end] for start, end in zip(starts, ends) if (end - start) > min_length]
    return sequences

def remove_outliers_median_filter(data, window=4):
    data = np.array(data, dtype=float)  # Ensure NumPy array with float type
    filtered_data = np.copy(data)  # Copy to avoid modifying original data
    half_window = window // 2
    for i in range(len(data)):
        # Define window range, ensuring it doesn't exceed bounds
        start = max(0, i - half_window)
        end = min(len(data), i + half_window + 1)
        # Extract local values in window
        local_values = data[start:end]
        # Check if the window contains at least one non-NaN value
        if np.all(np.isnan(local_values)):
            median_value = np.nan  # Keep NaN if no valid numbers
        else:
            median_value = np.nanmedian(local_values)  # Compute median ignoring NaNs
        # Replace only if the current value is not NaN
        if not np.isnan(data[i]):
            filtered_data[i] = median_value
    return filtered_data

# Remove uncertain location predictions (likelihood < 0.9)
df.iloc[:, 0] = df.apply(lambda row: row.iloc[0] if row.iloc[-1] > 0.9 else np.nan, axis=1)
df.iloc[:, 1] = df.apply(lambda row: row.iloc[1] if row.iloc[-1] > 0.9 else np.nan, axis=1)

X = df.iloc[:, 0]
Y = df.iloc[:, 1]

# Separate the individual's positions into x and y coordinates
individual_x = np.array(X.values)
individual_y = np.array(Y.values)

# Define when the mouse is on the cheeseboard (start)
for i, x in enumerate(individual_x):
    y = individual_y[i]
    if calculate_relative_distance(x, y, table_center_x, table_center_y) >= table_radius:
        individual_x[i] = np.nan
        individual_y[i] = np.nan

x_start = find_long_non_nan_sequences(individual_x)[0][0] # first value of the first long non nan sequence
y_start = find_long_non_nan_sequences(individual_y)[0][0] # first value of the first long non nan sequence
start_frame = np.where(individual_x == x_start)[0].item()
individual_x[:start_frame]=np.nan # remove any path before the real start
individual_y[:start_frame]=np.nan # remove any path before the real start

individual_x = remove_outliers_median_filter(individual_x)
individual_y = remove_outliers_median_filter(individual_y)

for i in range(len(individual_x)-1, 0, -1): # Find the last non-NaN value which is not isolated
    if not np.isnan(individual_x[i]) and not np.isnan(individual_x[i-1]):
        last_frame = i
        break
    
if timestamps_path.exists():
    start_time = timestamps.iloc[start_frame,1].item() / 1000
    end_time = timestamps.iloc[last_frame,1].item() / 1000
    duration_trial = end_time - start_time
else:
    duration_trial = (last_frame - start_frame) / frame_rate
print(f'Total trial duration: {round(duration_trial)} sec')

# Define when the mouse eats the reward (end)
found_reward_frame = np.nan
consecutive_count = 0
for i, (x, y) in enumerate(zip(individual_x, individual_y)):
    if calculate_relative_distance(x, y, reward_x, reward_y) <= reward_zone:
        consecutive_count += 1
    else:
        consecutive_count = 0  
    if consecutive_count > min_stay_at_reward:
        found_reward_frame = (i - min_stay_at_reward)
        break
    
if timestamps_path.exists():
    found_reward_time = timestamps.iloc[found_reward_frame,1].item() / 1000
    latency = (found_reward_time - start_time)
else:
    latency = (found_reward_frame - start_frame) / frame_rate
print(f'Latency to reward: {round(latency)} sec')

# Define the time spent inside the reward zone
time_spent_in_zone = 0

for i, (x, y) in enumerate(zip(individual_x, individual_y)):
    if calculate_relative_distance(x, y, reward_x, reward_y) <= reward_zone:
        time_spent_in_zone += 1

time_spent_in_zone = time_spent_in_zone / frame_rate
print(f'Time spent in reward zone: {round(time_spent_in_zone)} sec ({round(time_spent_in_zone / duration_trial * 100)} %)')

# Remove NaN values 
individual_x_filt = individual_x[~np.isnan(individual_x)]        
individual_y_filt = individual_y[~np.isnan(individual_y)]

# Define the number of times the mouse enters the reward zone
enter_reward_zone = 0
consecutive_count = 0
for i, (x, y) in enumerate(zip(individual_x_filt, individual_y_filt)):
    if calculate_relative_distance(x, y, reward_x, reward_y) <= reward_zone:
        if consecutive_count == 0:
            enter_reward_zone += 1
            consecutive_count = 1
    else:
        consecutive_count = 0   

print(f'Crossings in the reward zone: {enter_reward_zone}')

total_distance, distances = calculate_distance_run(individual_x[start_frame:last_frame], individual_y[start_frame:last_frame])
print(f"Total distance run: {round(total_distance)} cm")

distance_to_reward, distances_to_reward = calculate_distance_run(individual_x[start_frame:found_reward_frame], individual_y[start_frame:found_reward_frame])
print(f'Distance run to reward: {round(distance_to_reward)} cm')

print(f"Crossings per meter: {round(enter_reward_zone / (round(distance_to_reward) / 100), 2)}")

print(f"Average speed: {round(np.nanmean(distances_to_reward)/pixel_to_cm*frame_rate,2)} cm/s")

# Create the plot
fig, ax = plt.subplots(figsize=(3, 3)) 

# Plot individual positions over time
cmap = plt.get_cmap('gnuplot2')
norm = plt.Normalize(vmin=0, vmax=len(individual_x_filt))

for i in range(1, len(individual_x_filt)):
    ax.plot(individual_x_filt[i-1:i+1], individual_y_filt[i-1:i+1], color=cmap(norm(i)), linewidth=3)
#plt.plot(individual_x_filt, individual_y_filt, label="Individual's Path", color='b')

plt.scatter(x_start, y_start, color='black', s=100, label='Start')
plt.scatter(reward_x, reward_y, color='red', s=100, label='Reward')

# Draw reward zone circle
reward_circle = plt.Circle((reward_x, reward_y), reward_zone, color='r', fill=False)
plt.gca().add_patch(reward_circle)  

# Draw cheeseboard circle
table_circle = plt.Circle((table_center_x, table_center_y), table_radius, color='k', fill=False)
plt.gca().add_patch(table_circle) 

# Add labels and title
ax.set_aspect('equal')
ax.invert_yaxis()
plt.title(f'Mouse Path On Cheeseboard')
plt.xlabel('X Position')
plt.ylabel('Y Position')
plt.legend(loc='upper left')
plt.tight_layout()
plt.show()

Plot the spatial map for all cells + interactive Ca2+ trace

In [ ]:
# Set up selector object
discrete_slider = pn.widgets.DiscreteSlider(
    name="Choose unit", 
    options=[i for i in a.index],
    value=a.index[0]
)

next_unit_button = pn.widgets.Button(name='Next unit > ', button_type='primary')
previous_unit_button = pn.widgets.Button(name='< Previous unit')

# Define a callback function for the button
def nextunit_callback(event):
    position = np.where(a.index == discrete_slider.value)[0]
    position = position[0]
    nextunitvalue=a.index[position + 1] if position+2<=len(a) else a.index[0]
    discrete_slider.value = nextunitvalue
    
# Define a callback function for the button
def previousunit_callback(event):
    position = np.where(a.index == discrete_slider.value)[0]
    position = position[0]
    previousunitvalue=a.index [position - 1]
    discrete_slider.value = previousunitvalue

next_unit_button.on_click(nextunit_callback)
previous_unit_button.on_click(previousunit_callback)

# Define interactivity
@pn.depends(indexes=discrete_slider)
def calciumtrace(indexes):
    index = indexes
    position = np.where(a.index == index)[0]
    position = position[0]
    return hv.Curve((tsmini/1000, C[position, :]), label=f'Unit n° {index}').opts(ylim=(-0.5, 10), frame_height=200, color='red')

@pn.depends(indexes=discrete_slider)
def circlepath(indexes):
    index = indexes
    radius = 15
    num_points=100
    theta = np.linspace(0, 2*np.pi, num_points)
    position = np.where(a.index == index)[0]
    position = position[0]
    return hv.Path((a.iloc[position, 0] + radius * np.cos(theta), a.iloc[position, 1] + radius * np.sin(theta)), group='keep').opts(ylim=(0, 600), xlim=(0, 600), line_color='red', line_width=3) #

In [ ]:
output_size = 150
hv.output(size=int(output_size))

image = hv.Image(
    A.max("unit_id").compute().astype(np.float32).rename("A"),
    kdims=["width", "height"],
).opts(colorbar=False, invert_yaxis=False,cmap="Viridis")

alltraces=hv.NdOverlay({idx: hv.Curve((tsmini/1000, C[idx,:])).opts(frame_height=200, show_legend=False, color='black', alpha=0.2, xlabel='time (s)')
                       for idx in np.arange(len(C))})

start =  hv.VLine(start_time).opts(color='blue', line_width=2)
reward = hv.VLine(found_reward_time).opts(color='green', line_width=2)
starttxt = hv.Text(start_time, 9.5, 'Start').opts(text_color='blue')#, fontsize=7, text_align='start')
rewardtxt = hv.Text(found_reward_time, 9.5, 'Reward').opts(text_color='green')#, fontsize=7, text_align='end')

layout = pn.Column(pn.Row(image * hv.DynamicMap(circlepath),  # hv.DynamicMap(imgZoom)* hv.DynamicMap(circlepath),
            pn.Column(starttxt*rewardtxt*reward*start*alltraces * hv.DynamicMap(calciumtrace), discrete_slider, pn.Row(previous_unit_button, next_unit_button),       
                    ),
                    ))   

display(layout)

Downsample calcium traces and define eating time

In [ ]:
closest_start= find_closest_index_sorted(tsmini, start_time*1000)
closest_end= find_closest_index_sorted(tsmini, end_time*1000)
individual_x_nonan = remove_outliers_avg_filter(individual_x)
individual_y_nonan = remove_outliers_avg_filter(individual_y)

# Downsample calcium traces to webcam frame rate 
new_len = len(individual_x_nonan)  # Target length
old_indices = np.linspace(0, 1, C.values.shape[1])
new_indices = np.linspace(0, 1, new_len)
Cds_all = np.array([np.interp(new_indices, old_indices, row) for row in C.values])

total_distance, speed = calculate_distance_run(individual_x_nonan[start_frame:last_frame], individual_y_nonan[start_frame:last_frame])
distance_to_reward = np.sqrt((individual_x_nonan[start_frame:] - reward_x)**2 + (individual_y_nonan[start_frame:] - reward_y)**2)/pixel_to_cm

min_value, threshold, longest_sequence = detect_longest_lowest_sequence(distance_to_reward, np.nanstd(distance_to_reward)/3)
if longest_sequence:
    start_eating_frame, end_eating_frame, duration_eating = longest_sequence
    start_eating_frame=start_eating_frame + start_frame
    end_eating_frame=end_eating_frame + start_frame
else:
    start_eating_frame=np.nan
    end_eating_frame=np.nan
    duration_eating=np.nan

if timestamps_path.exists():
    start_eating_time = timestamps.iloc[start_eating_frame,1].item() / 1000
    end_eating_time = timestamps.iloc[end_eating_frame,1].item() / 1000
    duration_eating = end_eating_time - start_eating_time
else:
    start_eating_time = (start_eating_frame) / frame_rate
    end_eating_time = (end_eating_frame) / frame_rate
    duration_eating = (end_eating_frame - start_eating_frame) / frame_rate

Choose a neuron to plot

In [ ]:
nr=1

Plot neuron's activity during the trial

In [ ]:
plt.close('all')
plt.figure(figsize=[10,2])
#plt.plot(timestamps['Time Stamp (ms)'][start_frame:]/1000, zscore(individual_x_nonan[start_frame:]), 'g')
#plt.plot(timestamps['Time Stamp (ms)'][start_frame:]/1000, zscore(individual_y_nonan[start_frame:]), 'y')
plt.plot(timestamps['Time Stamp (ms)'][start_frame+1:last_frame]/1000, (speed)/max(speed), 'c', label='Mouse speed')
plt.plot(timestamps['Time Stamp (ms)'][start_frame:]/1000, (distance_to_reward)/max(distance_to_reward), 'm', label='Mouse distance to reward')

Cnr=C[nr,closest_start:closest_end]
normalized_C = (Cnr - np.min(Cnr)) / (np.max(Cnr.values) - np.min(Cnr.values)) if np.sum(Cnr)!=0 else Cnr
plt.plot(tsmini[closest_start:closest_end]/1000, normalized_C, 'k', label= f'Neuron #{nr}')
Call=C[:,closest_start:closest_end]
normalized_Cmean = (np.mean(Call, axis=0) - np.min(np.mean(Call, axis=0))) / (np.max(np.mean(Call, axis=0)) - np.min(np.mean(Call, axis=0))) 
plt.plot(tsmini[closest_start:closest_end]/1000, normalized_Cmean , 'k', alpha=0.2, label= f'Mean neuron activity')
plt.axvline(x=start_time, color='b', linewidth=2)
plt.axvline(x=found_reward_time, color='r', linewidth=2)
plt.axvline(x=start_eating_time, color='r', linestyle="dotted", linewidth=2)
plt.axvline(x=end_eating_time, color='r',linestyle="dotted", linewidth=2)
plt.text(start_time, plt.gca().get_ylim()[1], 'Start', fontsize=8, color='blue')
plt.text(found_reward_time, plt.gca().get_ylim()[1], 'Found reward', fontsize=8, color='red')
#plt.text(start_eating_time, plt.gca().get_ylim()[1], 'Started eating', fontsize=8, color='red')
plt.text(end_eating_time, plt.gca().get_ylim()[1], 'Stopped eating', fontsize=8, color='red')
plt.xlabel('time (s)')
plt.legend(frameon=False, bbox_to_anchor=(1, 1))
plt.tight_layout() 
plt.show()

Plot neuron's activity on the cheeseboard

In [ ]:
Cds_nr = Cds_all[nr,start_frame:last_frame]
x= individual_x_nonan[start_frame:last_frame]
y= individual_y_nonan[start_frame:last_frame]

# Plot circular environment
fig, ax = plt.subplots(figsize=(4, 4))
table_circle = plt.Circle((table_center_x, table_center_y), table_radius, color='k', fill=False)
plt.gca().add_patch(table_circle) 

# Normalize Cds_nr for colormap
if Cds_nr.min() != Cds_nr.max():
    norm = mcolors.Normalize(vmin=Cds_nr.min(), vmax=Cds_nr.max())
else:
    norm = mcolors.Normalize(vmin=0, vmax=1)
cmap = cm.jet

# Plot path with colormap
for i in range(len(x) - 1):
    color = cmap(norm(Cds_nr[i]))
    ax.plot([x[i], x[i+1]], [y[i], y[i+1]], color=color, linewidth=6)

# Add colorbar
sm = cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbar = plt.colorbar(sm, ax=ax)
plt.scatter(x_start, y_start, color='black', s=100, label='Start')
plt.scatter(reward_x, reward_y, color='red', s=100, label='Reward')
reward_circle = plt.Circle((reward_x, reward_y), reward_zone, color='r', fill=False)
plt.gca().add_patch(reward_circle)  
ax.invert_yaxis()
ax.set_aspect('equal')
plt.title(f'Neuron #{nr}')
plt.tight_layout()
plt.show()

Plot activity of each neurons on the cheeseboard

In [ ]:
nb_subplot=min(C.shape[0], 100)
rows = int(np.ceil(np.sqrt(nb_subplot)))  # Rows: ceil(sqrt(X))
cols = int(np.floor(np.sqrt(nb_subplot)))  # Columns: floor(sqrt(X))

# Create the figure with a 2x2 grid
fig, axs = plt.subplots(rows, cols, figsize=(15, 15))
axs = axs.flatten()
plt.tight_layout()

for nr in range(nb_subplot):

    Cds_nr = Cds_all[nr,start_frame:last_frame]
    x= individual_x_nonan[start_frame:last_frame]
    y= individual_y_nonan[start_frame:last_frame]

    # Plot circular environment
    table_circle = plt.Circle((table_center_x, table_center_y), table_radius, color='k', fill=False)
    axs[nr].add_patch(table_circle) 

    # Normalize Cds_nr for colormap
    if Cds_nr.min() != Cds_nr.max():
        norm = mcolors.Normalize(vmin=Cds_nr.min(), vmax=Cds_nr.max())
    else:
        norm = mcolors.Normalize(vmin=0, vmax=1)
    cmap = cm.jet

    # Plot path with colormap
    for i in range(len(x) - 1):
        color = cmap(norm(Cds_nr[i]))
        axs[nr].plot([x[i], x[i+1]], [y[i], y[i+1]], color=color, linewidth=3)

    # Add colorbar
    sm = cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])

    # Scatter start and reward points
    axs[nr].scatter(x_start, y_start,  marker='+', color='black', s=100, label='Start')
    axs[nr].scatter(reward_x, reward_y,  marker='+', color='red', s=100, label='Reward')

    # Customize plot
    axs[nr].invert_yaxis()
    axs[nr].set_aspect('equal')
    axs[nr].set_title(f'#{nr}', pad=0, loc='left')
    
    # Remove box (spines)
    for spine in axs[nr].spines.values():
        spine.set_visible(False)
    axs[nr].set_xticks([])  # No x-axis ticks
    axs[nr].set_yticks([])  # No y-axis ticks

# Adjust layout to avoid clipping
fig.subplots_adjust(wspace=0, hspace=0)
plt.show()

Average neuron activity on the cheeseboard

In [ ]:
Cds_all_mean=np.mean(Cds_all[:,start_frame:last_frame], axis=0)
x= individual_x_nonan[start_frame:last_frame]
y= individual_y_nonan[start_frame:last_frame]

# Plot circular environment
fig, ax = plt.subplots(figsize=(5, 5))
table_circle = plt.Circle((table_center_x, table_center_y), table_radius, color='k', fill=False)
plt.gca().add_patch(table_circle) 

# Normalize Cds_all_mean for colormap
norm = mcolors.Normalize(vmin=Cds_all_mean.min(), vmax=Cds_all_mean.max())
cmap = cm.jet

# Plot path with colormap
for i in range(len(x) - 1):
    color = cmap(norm(Cds_all_mean[i]))
    ax.plot([x[i], x[i+1]], [y[i], y[i+1]], color=color, linewidth=6)

# Add colorbar
sm = cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbar = plt.colorbar(sm, ax=ax)
plt.scatter(x_start, y_start, color='black', s=100, label='Start')
plt.scatter(reward_x, reward_y, color='red', s=100, label='Reward')
reward_circle = plt.Circle((reward_x, reward_y), reward_zone, color='r', fill=False)
plt.gca().add_patch(reward_circle)  
ax.invert_yaxis()
ax.set_aspect('equal')
plt.title(f'Average activity')
plt.tight_layout()
plt.show()

Average activity during specific behavior

In [ ]:
Cds_trial = Cds_all[:,start_frame:last_frame]
Cds_seeking = Cds_all[:,start_frame:found_reward_frame]
Cds_eating = Cds_all[:,start_eating_frame:end_eating_frame]

Cds_trial_m=np.mean(Cds_trial, axis=1)
Cds_seeking_m=np.mean(Cds_seeking, axis=1)
Cds_eating_m=np.mean(Cds_eating, axis=1)

# Compute mean and SEM
means = [np.mean(Cds_trial_m), np.mean(Cds_seeking_m), np.mean(Cds_eating_m)]
sems = [sem(Cds_trial_m), sem(Cds_seeking_m), sem(Cds_eating_m)]

# Bar plot
labels = ['Trial', 'Seeking', 'Eating']
x = np.arange(len(labels))

plt.figure(figsize=(6, 5))
plt.bar(x, means, yerr=sems, capsize=5, color=['blue', 'orange', 'green'], alpha=0.6)

"""
# Scatter individual points (jittered for better visibility)
for i, data in enumerate([Cds_trial_m, Cds_seeking_m, Cds_eating_m]):
    jitter = np.random.uniform(-0.1, 0.1, size=len(data))  # Add jitter for visibility
    plt.scatter(np.full_like(data, x[i]) + jitter, data, color='black', alpha=0.7, marker='.')
"""
plt.xticks(x, labels)
plt.ylabel('Calcium activity (a.u.)')
plt.title('Average neuron activity')

plt.show()
